In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import multiprocessing
from multiprocessing.pool import ThreadPool
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import keras
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from skimage import color
from skimage import io
from skimage import transform
import tensorflow as tf
# from tensorflow.keras.applications import MobileNetV2
# from tensorflow.keras.applications import MobileNet
# from tensorflow.keras.applications.mobilenet import preprocess_input
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

import tensorflow as tf
from tensorflow.python import debug as tfdebug
import os
import cv2
import scipy

Using TensorFlow backend.


In [2]:
images = np.load('all/train_images.npy', encoding="bytes")
label_arr = np.genfromtxt("./all/train_labels.csv",dtype='str', skip_header=1, delimiter = ',')
labels = pd.read_csv('all/train_labels.csv')
images[0]

array([0, array([0., 0., 0., ..., 0., 0., 0.])], dtype=object)

In [3]:
def resize_28(filteredImage):
    image, contours, hierarchy = cv2.findContours(filteredImage,cv2.RETR_LIST, \
                                   cv2.CHAIN_APPROX_SIMPLE)
    
    size=48
    if (contours != None):
        mx = (0,0,0,0)      # biggest bounding box so far
        mx_area = 0
    
        for cont in contours:
            x,y,w,h = cv2.boundingRect(cont)
            area = w*h
            if area > mx_area:
                mx = x,y,w,h
                mx_area = area
        x,y,w,h = mx
        roi=filteredImage[y:y+h,x:x+w]
        verti = size- roi.shape[0]
        hori = size- roi.shape[1]
        top = (size - roi.shape[0])//2
        bottom = size - top - roi.shape[0]
        left = (size - roi.shape[1])//2
        right = size - left - roi.shape[1]
        roi_copy = np.zeros((size,size))
        for i in range(top, roi.shape[0] + top):
            for j in range(left , roi.shape[1] + left):
                roi_copy[i][j] = roi[i-top][j-left]     
        filteredImage = roi_copy.reshape((size,size))
        
    else:       
        filteredImage = resize(filteredImage, (size,size))
    
    return filteredImage

In [4]:
def preProcessImage(image, cutoff=150, maxContours=10):
    image = np.uint8(image)
    im = np.uint8(image)
    red, thresh = cv2.threshold(im, cutoff, 255, 0)
    im2, contours, hierarchy= cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.zeros(im.shape, np.uint8)
    largest_contours = sorted(contours, key=cv2.contourArea, reverse=True)

    for ind, contour in enumerate(largest_contours[:5]):
        x, y, w, h = cv2.boundingRect(contour)
        mask[y:y+h, x:x+w] = 1
        
    filteredImage = cv2.bitwise_and(thresh, thresh, mask=mask)
    filteredImage[filteredImage>1] = 1
    filteredImage = filteredImage.reshape((image.shape))
    filteredImage = resize_28(filteredImage)
    
    
    
   
    return filteredImage

#     digits = ndimage.find_objects(filteredImage)
#     max_side = 0
#     max_ind = 0
#     for i in range(0,len(digits)):
#         bound = im[digits[i]]
#         if np.max(bound.shape) > max_side:
#             max_side = np.max(bound.shape)
#             max_ind = i
#     print (max_ind)
#     bound_rec = im[digits[max_ind]]

In [5]:
a = preProcessImage(images[5400][1].reshape(100, 100))

In [6]:
a.shape

(48, 48)

In [ ]:
# for i in range(10000):
#     if (True):
#         a = preProcessImage(images[i][1].reshape(100, 100))
#         #print(a.shape)
#         plt.imshow(a)
#         plt.figure()
#         plt.imshow(images[i][1].reshape(100, 100))
        
#         plt.show()

In [ ]:
a.shape

In [ ]:
label_arr[5400]

In [ ]:
plt.imshow(a)
plt.figure()
plt.imshow(images[5400][1].reshape(100, 100))
plt.show()

In [7]:
allData = pd.DataFrame(np.array(list(images[:,1]))).assign(label=labels['Category'])

In [8]:
num_classes = len(labels['Category'].unique())
lb = LabelBinarizer()
lb.fit(labels['Category'].unique())
valueCounts = labels['Category'].value_counts()

In [9]:
xTrain, xValid = train_test_split(allData, stratify=labels['Category'])
trainInds = xTrain.index
validInds = xValid.index
xTrainRaw = xTrain.drop('label', axis=1).values.reshape((xTrain.shape[0], 100, 100, 1))
xValidRaw = xValid.drop('label', axis=1).values.reshape((xValid.shape[0], 100, 100, 1))

In [10]:
yTrainString = labels.iloc[trainInds, 1].values
yValidString = labels.iloc[validInds, 1].values

In [11]:
yTrain = lb.transform(yTrainString)
yValid = lb.transform(yValidString)

In [12]:
pool = ThreadPool(multiprocessing.cpu_count())
xTrain = pool.map(preProcessImage, [xTrainRaw[i] for i in range(xTrainRaw.shape[0])])
xTrain = np.array(xTrain)
xValid = pool.map(preProcessImage, [xValidRaw[i] for i in range(xValidRaw.shape[0])])
xValid = np.array(xValid)
pool.close()
pool.join()

In [13]:
xTrain.shape

(7500, 48, 48)

In [16]:
def reshape_data(xTrain, size=48):
    Train_x = np.zeros((xTrain.shape[0], size, size, 1))
    for i in range(xTrain.shape[0]):
        Train_x[i] = xTrain[i].reshape(size, size, 1)
    return Train_x

In [17]:
Train_x = reshape_data(xTrain)
Valid_x = reshape_data(xValid)

In [18]:
Train_x.shape

(7500, 48, 48, 1)

In [ ]:
num = 1012
plt.imshow(Train_x[num][:,:,0])
plt.figure()
plt.imshow(xTrainRaw[num][:,:,0])
yTrainString[num]
plt.show()

In [ ]:
modelBasic = Sequential()
modelBasic.add(Dense(256, activation='relu', input_shape=(36, 36, 1)))
modelBasic.add(Dropout(0.25))

modelBasic.add(Dense(10, activation='relu'))
modelBasic.add(Dropout(0.25))
modelBasic.add(Flatten())

#modelBasic.add(Activation(tf.nn.softmax))

#modelBasic.add(Dense(num_classes, activation='softmax'))
modelBasic.compile(loss = keras.losses.categorical_crossentropy,
             optimizer = keras.optimizers.Adadelta(),
             metrics=['accuracy'])

In [ ]:
model6 = Sequential()
model6.add(Conv2D(128, kernel_size=(15, 15), padding='same', activation='relu', input_shape=(36, 36, 1)))
model6.add(BatchNormalization(momentum=0.99, epsilon=0.00001))
model6.add(MaxPooling2D(pool_size=(10, 10), strides=(7, 7), padding='same'))

model6.add(Conv2D(256, kernel_size=(5, 5), padding='same'))
model6.add(BatchNormalization(momentum=0.99, epsilon=0.00001))
model6.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'))

model6.add(Flatten())

model6.add(Dense(384, activation='relu'))
model6.add(Dense(192, activation='relu'))


model6.add(Dense(num_classes, activation='softmax'))
model6.compile(loss = keras.losses.categorical_crossentropy,
             optimizer = keras.optimizers.Adadelta(),
             metrics=['accuracy'])

In [ ]:
model = model6
model.summary()

In [ ]:
history = model6.fit(Train_x, yTrain, batch_size=batch_size,
                    epochs=epochs, verbose=1, 
                    validation_data=(Valid_x, yValid))

In [ ]:
model7 = Sequential()
model7.add(Conv2D(128, kernel_size=(15, 15), padding='same', activation='relu', input_shape=(28, 28, 1)))
model7.add(BatchNormalization(momentum=0.99, epsilon=0.00001))
model7.add(MaxPooling2D(pool_size=(10, 10), strides=(7, 7), padding='same'))

model7.add(Conv2D(256, kernel_size=(5, 5), padding='same'))
model7.add(BatchNormalization(momentum=0.99, epsilon=0.00001))
model7.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'))

model7.add(Flatten())

model7.add(Dense(384, activation='relu'))
model7.add(Dense(192, activation='relu'))


model7.add(Dense(num_classes, activation='softmax'))
model7.compile(loss = keras.losses.categorical_crossentropy,
             optimizer = keras.optimizers.SGD(lr=1e-3, momentum=0.0, decay=0.0, nesterov=True),
             metrics=['accuracy'])

In [ ]:
model = model7
model.summary()

In [ ]:
history = model7.fit(Train_x, yTrain, batch_size=batch_size,
                    epochs=epochs, verbose=1, 
                    validation_data=(Valid_x, yValid))

In [ ]:
model8 = Sequential()
model8.add(Conv2D(128, kernel_size=(15, 15), padding='same', activation='relu', input_shape=(36, 36, 1)))
model8.add(BatchNormalization(momentum=0.99, epsilon=0.00001))
model8.add(MaxPooling2D(pool_size=(10, 10), strides=(7, 7), padding='same'))

model8.add(Conv2D(256, kernel_size=(5, 5), padding='same'))
model8.add(BatchNormalization(momentum=0.99, epsilon=0.00001))
model8.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'))

model8.add(Flatten())

model8.add(Dense(384, activation='relu'))
model8.add(Dense(192, activation='relu'))


model8.add(Dense(num_classes, activation='softmax'))
model8.compile(loss = keras.losses.categorical_crossentropy,
             optimizer = keras.optimizers.Nadam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-8, schedule_decay=0.004),
             metrics=['accuracy'])

In [ ]:
model = model8
model.summary()

In [ ]:
history = model8.fit(Train_x, yTrain, batch_size=batch_size,
                    epochs=epochs, verbose=1, 
                    validation_data=(Valid_x, yValid))

In [ ]:
epochs = 20
batch_size = 128

model9 = Sequential()
model9.add(Conv2D(128, kernel_size=(15, 15), padding='same', activation='relu', input_shape=(36, 36, 1)))
model9.add(BatchNormalization(momentum=0.99, epsilon=0.00001))
model9.add(MaxPooling2D(pool_size=(10, 10), strides=(7, 7), padding='same'))

model9.add(Conv2D(256, kernel_size=(5, 5), padding='same'))
model9.add(BatchNormalization(momentum=0.99, epsilon=0.00001))
model9.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'))

model9.add(Flatten())

model9.add(Dense(384, activation='relu'))
model9.add(Dense(192, activation='relu'))


model9.add(Dense(num_classes, activation='softmax'))
model9.compile(loss = keras.losses.categorical_crossentropy,
             optimizer = keras.optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0),
             metrics=['accuracy'])

In [ ]:
model = model9
model.summary()

In [ ]:
history = model9.fit(Train_x, yTrain, batch_size=64,
                    epochs=64, verbose=1, 
                    validation_data=(Valid_x, yValid))

In [19]:
# Model parameters
BATCH_SIZE = 64
EPOCHS = 20
LEARNING_RATE = 0.001
HEIGHT = 48
WIDTH = 48
CHANNEL = 1

model10 = Sequential()

model10.add(Conv2D(50, kernel_size=(5,5),padding='Same', activation='relu', input_shape=(HEIGHT, WIDTH, CHANNEL)))
model10.add(Conv2D(50, kernel_size=(5,5),padding='Same', activation='relu'))
model10.add(MaxPool2D(pool_size=(2,2)))
model10.add(Dropout(0.25))

model10.add(Conv2D(100, kernel_size=(3,3),padding='Same', activation='relu'))
model10.add(Conv2D(100, kernel_size=(3,3),padding='Same', activation='relu'))
model10.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model10.add(Dropout(0.25))

model10.add(Flatten())

model10.add(Dense(256, activation='relu'))
model10.add(Dropout(0.5))

model10.add(Dense(num_classes, activation = "softmax"))

optimizer = optimizers.adam(lr=LEARNING_RATE)
model10.compile(optimizer=optimizer , loss="categorical_crossentropy", metrics=["accuracy"])

In [20]:
history = model10.fit(Train_x, yTrain, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, verbose=1, 
                    validation_data=(Valid_x, yValid))

Train on 7500 samples, validate on 2500 samples
Epoch 1/20
7500/7500 [==============================] - 148s 20ms/step - loss: 2.5176 - acc: 0.3101 - val_loss: 1.8142 - val_acc: 0.5072
Epoch 2/20
7500/7500 [==============================] - 150s 20ms/step - loss: 1.7613 - acc: 0.5169 - val_loss: 1.4205 - val_acc: 0.6052
Epoch 3/20
7500/7500 [==============================] - 147s 20ms/step - loss: 1.4815 - acc: 0.5843 - val_loss: 1.2711 - val_acc: 0.6512
Epoch 4/20
7500/7500 [==============================] - 146s 19ms/step - loss: 1.2831 - acc: 0.6357 - val_loss: 1.1661 - val_acc: 0.6720
Epoch 5/20
7500/7500 [==============================] - 141s 19ms/step - loss: 1.1533 - acc: 0.6701 - val_loss: 1.1523 - val_acc: 0.6820
Epoch 6/20
7500/7500 [==============================] - 143s 19ms/step - loss: 1.0280 - acc: 0.7024 - val_loss: 1.1343 - val_acc: 0.6944
Epoch 7/20
7500/7500 [==============================] - 144s 19ms/step - loss: 0.9444 - acc: 0.7259 - val_loss: 1.0626 - val_acc: 

In [21]:
#added by Yang
TEST_DIR ='all/test_images.npy'
TESTSET=np.load(TEST_DIR,encoding='bytes')
Testset=TESTSET[:,1]
Testset=[Testset[i] for i in range(len(Testset))]
Testset=np.array(Testset)
Testset=Testset.reshape((Testset.shape[0], 100, 100, 1))
pool = ThreadPool(multiprocessing.cpu_count())
xTest = pool.map(preProcessImage, [Testset[i] for i in range(Testset.shape[0])])
xTest = np.array(xTest)
pool.close()
pool.join()
Test_x = reshape_data(xTest)


In [41]:
#added by Yang
predictions = model10.predict_classes(Test_x)

In [43]:
#added by Yang
TRAIN_LABEL_DIR='all/train_labels.csv'
Trainlabel=pd.read_csv(TRAIN_LABEL_DIR)
train_labels, train_uniques=pd.factorize(Trainlabel['Category'])
dictionary=dict(zip(range(0,32), train_uniques))

In [44]:
predictions=pd.DataFrame(predictions)
predictions[0]=[dictionary[item] for item in predictions[0]]
predictions=predictions[0]


In [45]:
print(predictions)


0              pear
1          sailboat
2            pillow
3            pillow
4             rifle
5            pencil
6            pillow
7         moustache
8              pear
9             empty
10       paintbrush
11             pear
12           pencil
13         squiggle
14        pineapple
15             nose
16           pencil
17         squiggle
18            mouth
19           pencil
20           pillow
21            spoon
22      screwdriver
23         squiggle
24           peanut
25            apple
26         scorpion
27            panda
28            mouth
29       rhinoceros
           ...     
9970    screwdriver
9971           sink
9972      pineapple
9973     skateboard
9974          apple
9975      moustache
9976           sink
9977          rifle
9978      pineapple
9979       scorpion
9980       squiggle
9981          apple
9982          spoon
9983          skull
9984           pear
9985          apple
9986           pear
9987           pear
9988          rifle


In [46]:
#added by Yang
def save_to_csv(name, result):
    import csv
    with open(name,'w',newline='') as f:
        fieldnames=['Id','Category']
        thewriter=csv.DictWriter(f,fieldnames=fieldnames)
        thewriter.writeheader()
        for i in range(len(result)):
            thewriter.writerow({'Id':i,'Category':result[i]})

In [47]:
#added by Yang
save_to_csv('prediction.csv',predictions)


In [ ]:
# Model parameters
BATCH_SIZE = 64
EPOCHS = 20
LEARNING_RATE = 0.001
HEIGHT = 36
WIDTH = 36
CHANNEL = 1

model12 = Sequential()

model12.add(Conv2D(50, kernel_size=(5,5),padding='Same', activation='relu', input_shape=(HEIGHT, WIDTH, CHANNEL)))
model12.add(Conv2D(50, kernel_size=(5,5),padding='Same', activation='relu'))
model12.add(MaxPool2D(pool_size=(2,2)))
model12.add(Dropout(0.25))

model12.add(Conv2D(100, kernel_size=(3,3),padding='Same', activation='relu'))
model12.add(Conv2D(100, kernel_size=(3,3),padding='Same', activation='relu'))
model12.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model12.add(Dropout(0.25))

model12.add(Conv2D(200, kernel_size=(3,3),padding='Same', activation='relu'))
model12.add(Conv2D(200, kernel_size=(3,3),padding='Same', activation='relu'))
model12.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model12.add(Dropout(0.25))

model12.add(Flatten())

model12.add(Dense(256, activation='relu'))
model12.add(Dropout(0.5))

model12.add(Dense(num_classes, activation = "softmax"))

optimizer = optimizers.adam(lr=LEARNING_RATE)
model12.compile(optimizer=optimizer , loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
history = model12.fit(Train_x, yTrain, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, verbose=1, 
                    validation_data=(Valid_x, yValid))

In [ ]:
predictions = model12.predict(Valid_x)

In [ ]:
true = 0
false = 0
false_class = [0]*31
for i in range(2500):
    if np.argmax(yValid[i]) == np.argmax(predictions[i]):
        true += 1
    else:
        #print("True class:", np.argmax(yValid[i]))
        #print("False prediction:", np.argmax(predictions[i]))
        false_class[np.argmax(yValid[i])] += 1

In [ ]:
print (predictions.shape)

In [ ]:
def reshape_data_2(xTrain, size=36):
    Train_x = np.zeros((xTrain.shape[0], size, size))
    for i in range(xTrain.shape[0]):
        Train_x[i] = xTrain[i].reshape(size, size)
    return Train_x

In [ ]:
Train_x = reshape_data_2(xTrain)
Valid_x = reshape_data_2(xValid)

In [ ]:
nb_units = 50

model11 = Sequential()

# Recurrent layers supported: SimpleRNN, LSTM, GRU:
model11.add(SimpleRNN(nb_units,
                    input_shape=(36, 36)))

# To stack multiple RNN layers, all RNN layers except the last one need
# to have "return_sequences=True".  An example of using two RNN layers:
#model.add(SimpleRNN(16,
#                    input_shape=(img_rows, img_cols),
#                    return_sequences=True))
#model.add(SimpleRNN(32))

model11.add(Dense(units=num_classes))
model11.add(Activation('softmax'))

model11.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model11.summary())

In [ ]:
Train_x = reshape_data(xTrain)
Valid_x = reshape_data(xValid)

In [ ]:
batch_sz = 128
epochs = 300
filter_sizes = [18,18,18]
hidden_sz = 128
model = Sequential()
model.add(Conv2D(18, (5, 5), padding='same', input_shape=(36,36,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))

for fs in filter_sizes:
    model.add(Conv2D(fs, (1,1)))
    model.add(Conv2D(fs, (3,3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Conv2D(fs, (1,1)))
    # model.add(Conv2D(fs, (3,3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(hidden_sz))
# model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

opt = keras.optimizers.Adam(lr=5e-3)
model.compile(loss='categorical_crossentropy',optimizer=opt, metrics=['accuracy'])

In [ ]:
history = model.fit(Train_x, yTrain, batch_size=batch_sz,
                    epochs=epochs, verbose=1, 
                    validation_data=(Valid_x, yValid))